### References

[존이 Blog](http://blog.naver.com/PostView.nhn?blogId=mykepzzang&logNo=220836321999) - PDF, CDF, marginal pdf, conditional distribution

[알 Blog](http://blog.daum.net/gongdjn/57) - Joint PDF

[문일철 교수님의 머신러닝 유튜브 강의](https://www.youtube.com/watch?v=bru-vqk1tYI&list=PLbhbGI_ppZIRPeAjprW9u9A46IJlGFdLn&index=39) - Gaussian process

[Mad for Simplicity Blog](http://enginius.tistory.com/489) - stochastic process (random process)

[SanghyukChun's Blog](http://sanghyukchun.github.io/99/) - Practical Bayesian Optimization of Machine Learning

[Mathematics Q&A](https://math.stackexchange.com/questions/885410/covariance-matrix-of-various-x-y-z-cartesian-coordinates) - covariance matrix


# Practical Bayesian Optimization of Machine Learning - 2012

### 기존 방법

기존에 최적의 hyperparameter를 찾기 위해서는 

1. grid search
2. random search

를 주로 사용하였습니다.

learning rate, num_hidden_layers, num_hidden_units, dropour_rate

를 예로 들겠습니다.

##### grid search

grid search는 naive 한 방법입니다. 모든 경우의수를 살펴보는 것이죠.

사용자가 처음에 찾고 싶은 파라미터 범위를 정합니다.

``` python
learning_rate = [0.1, 0.01, 0.001, 0.0001]   
num_hidden_layers = [2, 3]
num_hidden_units = [64, 128, 256]
dropout_rate = [0.3, 0.4, 0.5, 0.6]          
```
범위를 위와 같이 정한다면 hyperparameter들의 모든 조합은 4 * 2 * 3 * 4 로 96가지 입니다.

96가지 경우를 모두 시행해보고 가장 좋은 parameter set을 구합니다.

##### random search

파리미터들 중에는 결과에 큰 영향을 주는 파라미터가 있는 반면에 결과에 영향을 많이 미치지 않는 파라미터가 있습니다.

random search는 랜덤으로 표본을 추출해서 결과에 큰 영향을 주는 parameter를 확률적으로 더 많이 살펴볼 수 있다고 합니다.

결과도 grid search 보다 훨씬 좋게 나왔었죠.

![image.png](./imgs/grid_random.png)

이를 practical 하게 사용하기 어려운 3가지 이유가 있는데

1. kernel_function, acquisition function 등 어떤 모델을 선택하냐에 따라 성능이 크게 바뀝니다.
2. bayesian optimization 자체도 hyperparameter가 있습니다.
3. Squential update를 해야하기 때문에 parallelization 이 되지 않습니다.

이 논문은 

1. 경험적으로 적절한 kernel function과 acquisition function을 제안합니다.
2. bayesian optimization 의 hyperparameter를 Markov Chain Monte Carlo로 풀 수 있는 fully bayesian approach를 통해 전체 optimization에서 한번에 계산할 수 있는 방법을 제안합니다.
3. Markov Chain Monte Carlo로 풀 수 있는 theoretically tractable parallelized bayesian optimization 을 제안합니다.


### Bayesian Optimization

Bayesian optimization은 다음과 같은 optimization 문제를 푸는 방법론 중 하나 입니다.

![image.png](./imgs/bayes_opt_func.png)

이때, X는 유한집합이고, f(x)는 input을 넣었을 때 output이 무엇인지만 알 수 있는 black box function이라 가정합니다.

f(x)가 input을 넣어서 output을 확인하는 것 자체가 cost가 많이 드는 expensive black-box function일 때, bayesiam optimization을 많이 사용합니다.

Bayesian optimization은 다음과 같은 방식으로 작동합니다.

1. 지금까지 관측된 데이터(input, output)들을 통해, f(x)를 GP를 이용하여 estimation 한다.
2. f(x)를 더 정밀하게 예측하기 위해 다음으로 관측할 지점을 선택한다. - 가장 큰 분산이 있는 지점
3. stopping criteria 가 만족될 때까지 새로 관측한 데이터를 이용해 1, 2를 반복한다.

##### example

빨간색 점선은 우리가 찾으려고 하는 f(x) 를 나타냅니다.

까만색 실선은 지금까지 관측한 데이터를 바탕으로 우리가 예측한 estimated function 입니다. 

까만선 주변에 있는 회색 영역은, function f(x)가 존재할 confidence bound(function의 분산)입니다.

밑에 있는 EI(x)는 acquisition function을 의미합니다.

![image.png](./imgs/GP1.png)

지금까지 관측한 데이터를 바탕으로, (acquisition function 값이 제일 큰) 점이 찍힌 부분을 관측하는 것이 가장 좋습니다.

![image.png](./imgs/GP2.png)

관측 후, estimation function를 업데이트 합니다.

acquisition function을 다시 구해 보고 가장 큰 값이 있는 point를 관측합니다.

![image.png](./imgs/GP3.png)

acquisition function은 kernel function과 같이 여러가지 함수가 존재합니다. 보통 가장 큰 분산이 있는 지점을 선택합니다.

결국 정리하자면

1. y 값을 loss라 두고 수많은 요인이 x 값인 고차원의 함수에서 loss가 가장 작을 때의 x값(hyperparameter) 들을 찾아내는 것이 목표입니다.
2. 이를 효율적으로 찾아내는 방법이 bayesian distribution 입니다.
3. 가장 먼저 랜덤으로 몇번의 실험을 해봅니다. 그러면 x 값들에 따른 loss를 구할 수 있지요.
4. 함수를 고차원에서 예측하는 기법이 Gaussian Process Regression 기법입니다.
5. acquisition function 을 이용하여 데이터를 업데이트하고 다시 GP를 돌리는 식으로 진행하여 함수를 추측해 내고
6. 가장 작은 수를 찾아냅니다.

# 그래서 GP를 어떻게 하는데?

### Requirements

1. PDF, CDF, Joint PDF, Marginal PDF, Conditional distribution
2. Mapping function (== kernel trick)
3. Kernel function
4. Gaussian process regression
5. Bayesian Optimization + acquisition function
6. Markov chain Monte Carlo

### PDF, CDF, Joint PDF, Marginal PDF, Conditional distribution

##### PDF

PDF 는 Probability Density Function의 약자입니다.

한글로 하면 확률 밀도 함수 입니다.

아래와 같은 특징이 있습니다.

![image.png](./imgs/pdf.png)

![image.png](./imgs/pdf2.png)

즉 함수의 a 부터 b 까지의 넓이를 구하면 그것이 구간에 대한 확률이 됩니다.

##### CDF

다음으로 CDF는 Cumulative Distribution Function으로 누적 분포 함수입니다.

아래와 같은 특징을 지닙니다.

![image.png](./imgs/cdf.png)

그림을 그려보면 아래와 같고

![image.png](./imgs/cdf2.png)

이런 특징도 지닙니다.

![image.png](./imgs/cdf3.png)

##### Joint PDF

두 개 이상의 확률 변수에 대한 분포를 Joint PDF 라 합니다. 

결합확률밀도 함수 입니다.

![image.png](./imgs/joint_pdf.png)

##### Marginal PDF

주변확률밀도 함수 입니다.

Joint PDF 에서 한 가지 확률 변수의 분포만을 구하는 방법입니다. 

Joint PDF 를 하나의 확률 변수로 표기 가능하도록 적분을 시키면 됩니다.

이러한 작업을 marginalize라 부릅니다.

![image.png](./imgs/marginal_pdf.png)

##### Conditional distribution

조건부 분포입니다. 

Marginal PDF를 이용하여 구해냅니다.

![image.png](./imgs/conditional_distribution.png)


### Mapping function

pi() 함수를 이용해 basis를 확장하는 함수입니다.

이를 통해 linear 하지 않은 문제를 linear 하게 바꿔줄 수 있습니다.

예를 들어 대표적으로 XOR 문제는 non-linear 한 문제입니다.

![image.png](./imgs/mapping_func.png)

하지만 위와 같은 pi() mapping function (basis function) 을 통해 dimension 확장을 꾀할 수 있고 linear 한 문제로 바꿔 줄 수 있습니다.

이를 하나의 식으로 나타낸다면 다음과 같이 나타낼 수 있습니다.

T는 transpose 를 의미합니다.

![image.png](./imgs/linear_regr_basis.png)

예를 들면, 아래과 같이 말입니다.

![image.png](./imgs/linear_regr_basis2.png)

또는 이렇게도요. 

![image.png](./imgs/design_mat3.png)

단 큰 파이를 사용하였는데 큰 파이는 아래와 같이 정의 됩니다.

![image.png](./imgs/design_mat.png)

X와 Y의 데이터셋을 정의했을 때 n은 데이터의 개수, k는 확장시킬 basis의 수 입니다.

![image.png](./imgs/design_mat2.png)

여기서 이 큰 파이를 design matrix 라고 부릅니다.

이제 W 값을 특정 값이 아닌 확률적 분포를 따라서 나온 값이라 생각해 봅시다.

![image.png](./imgs/normal_dist.png)

W 가 이런 normal distribution 을 따르기 때문에 Y 또한 확률적인 distribution 을 따릅니다. 

여기서 평균은 0, 분산은 covariance matrix 형태로 나타냅니다.

또한 데이터 간 correlation (연관성) 이 없다고 가정하였습니다.

covariance matrix 는 아래와 같은 형태 입니다.

![image.png](./imgs/covariance_mat.png)

따라서 Y의 평균과 분산을 구하면 아래와 같습니다.

![image.png](./imgs/predict_Y.png)

즉, K 라는 matrix를 아래와 같이 정의한다면 P(Y) 확률분포는 가장 아래 그림과 같이 표현할 수 있습니다.

![image.png](./imgs/kernel_func_lec5.png)

이해가 안 가실 것 같아서 추가 예시를 드립니다.

![image.jpg](./imgs/mine.jpg)

### Kernel function

커널함수는 벡터를 고차원으로 보낸 값들의 내적으로 정의가 됩니다.

K(x, y) = pi(x) * pi(y)

특이한 점은, mapping function 의 바뀌는 차원수가 n 이라고 쳤을 때,

K(x, y) = pi(x) * pi(y)

      = pi(x * y)

가 성립이 된다는 것입니다.

먼저 낮은 차원에서 내적을 하고 mapping function을 거쳐도 값이 똑같은 것이 특징입니다.

즉, 단순한 낮은차원에서의 벡터 연산을 통해 높은 차원의 값 성질들을 알아낼 수 있습니다.

pi() = [x1^2, sqrt(2) \* x1 \* x2, x2^2]^T 일 때 증명입니다.

![image.png](./imgs/polynomial_kernel.png)

아래는 이런 성질을 이용한 여러가지 커널 함수들을 나타낸 것입니다.

가우시안 커널 함수에서는 차원의 dimension에 관한 값이 식에 없는데, 

가우시안 커널 함수가 무한대 차원으로 갈 때 다음과 같은 식이 된다는 것을 증명했다고 합니다.

![image.png](./imgs/kernel_functions.png)

### Gaussian process

Gaussian process 는 어떤 함수 f(x) 를 이미 알고 있는 표본을 통해 추정하는 방법입니다.

예를 들어, 다음과 같은 함수가 있습니다.

y = sinc(x)

![image.png](./imgs/sinc_func.png)

싱크함수라고 불리우는 녀석입니다.

이 녀석을 기준으로 gaussian noise 를 50개 뽑았습니다.

![image.png](./imgs/sinc_sample50.png)

이 샘플 50개를 이용하여 원래의 sinc 함수를 추론해내는 가장 간단한 방법은 moving average 기법입니다.

time window (구간)를 설정하고 그 구간에서의 mean(평균값)과 variance(분산값)을 구해냅니다.

아래와 같은 형태가 되겠지요.

![image.png](./imgs/sinc_moving_avg_basic.png)

위 이미지에서는 time 윈도우를 [-1/5 파이, 1/5 파이] 로 설정하였습니다.

하지만 결과가 만족스럽지는 않습니다.

time 윈도우 구간을 늘린다면 너무 smooth 한 그래프가 예측이 될 것이고

time 윈도우 구간을 줄인다면 너무 흔들리는 그래프가 예측이 될 것입니다.

그래서 mean 과 variance 를 구할 때 가까운 곳에 있는 데이터에 더 많은 weight를 주고 멀리 떨어진 데이터에 더 적은 weight를 주었더니 아래와 같은 결과가 나왔습니다.

![image.png](./imgs/sinc_moving_avg_weighted1.png)

훨씬 결과가 깔끔합니다.

이 함수를 미분 가능하도록 exponenetial을 사용하였습니다.

![image.png](./imgs/kernel_squared_exp.png)

L이 작을수록 standard deviation 이 작은 형태의 bell 모양 함수 형태가 나올 것이고

L이 클 수록 standard deviation 이 큰 형태의 bell 모양 함수 형태가 나올 것입니다.

함수를 사용하였을 때의 결과는 아래와 같습니다.

![image.png](./imgs/squared_exp_L5.png)

분명 결과가 좋습니다. 하지만 결국 L이라는 hyperparameter 에 따라 함수가 결정이 되기 때문에 아쉽습니다.

이 함수를 좀 더 복잡하게 GP 형태로 만들어 볼 것입니다.

사실 위쪽에서 Mapping Function을 보며 구한 W distribution에 대한 Y distribution 값은 아래 이미지에서의 중간 값. error_free value 입니다.

![image.png](./imgs/what_we_found.png)

![image.png](./imgs/what_we_found2.png)

구하고자 하는 것은 실제 세계에서 오차를 포함하며 관측된 값. 즉 첫 번째 값입니다.

이를 표현한다면 다음과 같습니다.

![image.png](./imgs/pty.png)

identity matrix가 붙는 것은 error가 independent하다는 것입니다.

그러므로 P(T)의 확률분포는 다음과 같습니다.

![image.png](./imgs/pt.png)

이 값을 풀기 위해 marginal gaussian pdf 와 conditional distribution을 이용합니다.

간단히 설명드리면 P(Y, T) 분포에서 marginalize 시켜 P(T)에 대한 함수로 바꿉니다.

유도과정이 꽤 복잡해 생략하겠습니다. 궁금하신 분들은 문일철 교수님 강의를 보시면 되겠습니다.

유도 결과는 다음과 같습니다.

![image.png](./imgs/pt2.png)

이 때, kernel 값은 다음과 같습니다. 

![image.png](./imgs/kernel_func_fin.png)

또한 kernel의 세타값에 따라 모든 kernel 함수의 형태가 나올 수가 있습니다.

![image.png](./imgs/samples_kernel.png)


이제 우리가 하고 싶은 것은 과거의 값(관측한 값)을 통해 미래의 값을 알아내고 싶습니다.

즉, 이 값을 구하고 싶은 것이지요.

![image.png](./imgs/past_future.png)

이를 수식화하고 정리하면 regression을 수행한 결과가 나옵니다.

유도과정은 생략합니다.

![image.png](./imgs/regr_result.png)



여기에서 있는 5개의 hyperparameter를 kernel hyperparameter라 부르고 이것은 따로 학습을 시켜주어야 합니다.

학습을 시킬 때 negative likelihood 를 만들고 최소화 합니다.

어쨌든 모델에 대한 kernel function 자동화가 가능하였고 최적의 regr을 찾을 수 있다고 합니다.

이것이 GP 입니다.

GP regression은 scikit learn에서 제공해 주고 있습니다.

# 적용하기

https://github.com/fmfn/BayesianOptimization

이 코드를 보면 GP를 모두 SKlearn에서 제공해 주고 손쉽게 optimization을 구할 수 있는 것으로 보입니다.

코드를 조금만 적용하면 현업에 적용할 수 있는 것으로 보이는데 제가 너무 설명을 어렵게 해서 사용할 생각도 안들게 해버린 것 같네요.

꼭 코드 한 번 보세요 간단합니다.